In [ ]:
import pyemu
import conda_scripts.load_datasets as lsd
import os
import pandas as pd
import numpy as np
import conda_scripts
import forward_run
import matplotlib.pyplot as plt
import conda_scripts.plot_help as ph
from conda_scripts import sv_budget
import matplotlib as mpl
import numpy as np
import flopy
import geopandas as gpd
import xarray as xr
import rioxarray as rx

In [ ]:
workspace = lsd.model_info.get_mod('son', True)
workspace

In [ ]:
main = 'postprocess'
fold =  'pestaws_v1'

out_folder = os.path.join(main, fold)
if not os.path.exists(out_folder):
    os.makedirs(out_folder, exist_ok =True)

In [ ]:
ml = conda_scripts.sv_budget.load_sv_model.get_model(workspace)

In [ ]:
import xarray as xr
import numpy as np
from pyproj import Transformer



In [ ]:
ml.update_modelgrid()

In [ ]:
xl = ml.modelgrid.get_xcellcenters_for_layer(0)
yl = ml.modelgrid.get_ycellcenters_for_layer(0)

yl

In [ ]:
shallow = rx.open_rasterio(r"C:\GSP\waterlevel\GIS\hydro_experiment\maps__v3_SON_allmodmonths_to_20240808\SON_1970_Fall_Shallow.tif")


deep = rx.open_rasterio(r"C:\GSP\waterlevel\GIS\hydro_experiment\maps__v3_SON_allmodmonths_to_20240808\SON_1970_Fall_Deep.tif")


In [ ]:
zone = ml.bas6.ibound.array[1]
zone
plt.imshow(zone)

In [ ]:
import cartopy.crs as ccrs
import contextily as ctx

In [ ]:
da = xr.DataArray(
    data=xl,
    dims=[ "y",'x'],
    coords=dict(
        x=([ "y",'x'], xl),
        y=([ "y",'x'], yl),
        # time=time,
        # reference_time=reference_time,
    ),
    attrs=dict(
        description="Ambient temperature.",
        units="degC",
    ),
)
da

ar_shal= shallow.isel(band = 0).interp(da.coords)


ar_deep = deep.isel(band = 0).interp(da.coords)

plt.figure(dpi = 250, figsize = (6,8))
ax = plt.subplot(1, 1, 1, projection=ccrs.epsg(2226))

ar_shal.plot.contourf(ax =ax,  levels = np.arange(0, 1000, 50), alpha = .5)
ar_shal.plot.contour(ax =ax,  levels = np.arange(0, 1000, 50))
ctx.add_basemap(ax, crs = 2226)

plt.figure(dpi = 250, figsize = (6,8))
ax = plt.subplot(1, 1, 1, projection=ccrs.epsg(2226))

ar_deep.plot.contourf(ax =ax,levels = np.arange(0, 1000, 50), alpha = .5)
ar_deep.plot.contour(ax =ax,levels = np.arange(0, 1000, 50))
ctx.add_basemap(ax, crs = 2226)

plt.figure(dpi = 250, figsize = (6,8))
ax = plt.subplot(1, 1, 1, projection=ccrs.epsg(2226))
(ar_shal - ar_deep).plot.contour(ax =ax,levels = np.arange(-50, 51, 10))
ctx.add_basemap(ax, crs = 2226)

In [ ]:
ar_shal.data[zone==0] = -999
ar_deep.data[zone==0] = -999

In [ ]:
for lay in range(1,7):
    if lay<3:
        np.savetxt(os.path.join(workspace,'init_heads', f"init_heads_lay{lay}.dat"), ar_shal.data, fmt = "%.0f")
    else:
        np.savetxt(os.path.join(workspace,'init_heads', f"init_heads_lay{lay}.dat"), ar_deep.data, fmt = "%.0f")